In [1]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import datetime
import json
import psycopg2
import math
import numpy as np
import bisect

# load the credentials from the JSON file
with open('config/credentials.json') as f:
    credentials = json.load(f)

connection_string = f"postgresql://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database_name']}"

# create the engine with the connection string
engine = create_engine(connection_string)


C:\Users\domin\anaconda3\envs\car_sharing_env\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
### pre-processing of reservation table: ###
# reservationfrom_discrete: discretize reservationfrom to 15 minutes 
# reservationto_discrete: discretize reservationto to 15 minutes 
# syscreatedate_discrete: discretize syscreatedate to 15 minutes 
# time_booking_to_bookingstart: time difference between syscreatedate and reservationfrom, in 15 min units
# reservation_duration: time difference between reservationfrom_discrete and reservationto_discrete, in 15 min units
# syscreatedate_daytime: time of the day of syscreatedate, in 15 min units
# reservationfrom_daytime: time of the day of reservationfrom, in 15 min units
# reservationto_daytime: time of the day of reservationto, in 15 min units
# booking_trip_same_day: boolean wether a trip was reserved on the same day as the trip or not

sql = """WITH reservations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_firststart_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_lastend_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.reservation 
)
SELECT *, (drive_km/range)*100 AS required_soc,
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM reservations JOIN msc_2023_dominik.vehicle_information USING(vehicle_no) 
ORDER BY reservationfrom 
"""

data = pd.read_sql(sql, engine)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete']).dt.date 
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom']).dt.date 
data["drive_firststart_discrete_date"] = pd.to_datetime(data['drive_firststart']).dt.date 
data["booking_trip_same_day"] = data["reservationfrom_discrete_date"] == data["syscreatedate_discrete_date"] 
data["syscreatedate_daytime_endofday"] = data["syscreatedate_daytime"]
data["reservationfrom_daytime_endofday"] = data["reservationfrom_daytime"]
data["reservationto_daytime_endofday"]  = data["reservationto_daytime"] 

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime"] = 0

cond = data["reservationfrom_discrete_date"] <= data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = 96

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] + pd.Timedelta(days=1)
data.loc[cond, "syscreatedate_daytime_endofday"] = 0

cond = data["reservationfrom_daytime_endofday"] == 0
data.loc[cond, "reservationfrom_daytime_endofday"] = 96


cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = data["reservationfrom_daytime_endofday"]


cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

cond = data["required_soc"] < 0
data.loc[cond, "required_soc"] = 0

cond = data["required_soc"] > 100
data.loc[cond, "required_soc"] = 100

data.to_sql("reservations_no_service_discrete", engine, schema="msc_2023_dominik", if_exists='replace')
data.head()

# Service reservations
sql = """WITH reservations AS (
SELECT *,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(syscreatedate, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as syscreatedate_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationfrom, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationfrom_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_firststart, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_firststart_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(drive_lastend, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as drive_lastend_discrete,
(timestamp 'epoch' + (((EXTRACT(epoch FROM TO_TIMESTAMP(reservationto, 'YYYY-MM-DD HH24:MI:SS.MS'))::int + 450) / 900) * 900) * INTERVAL '1 second') as reservationto_discrete
FROM mobility.service_reservation 
)
SELECT *, (drive_km/range)*100 AS required_soc,
EXTRACT(epoch FROM (reservationfrom_discrete - syscreatedate_discrete)) / 900 AS time_booking_to_bookingstart,
EXTRACT(epoch FROM (reservationto_discrete - reservationfrom_discrete)) / 900 AS reservation_duration
FROM reservations JOIN msc_2023_dominik.vehicle_information USING(vehicle_no) 
ORDER BY reservationfrom 
"""

data = pd.read_sql(sql, engine)
data["syscreatedate_daytime"] = data.syscreatedate_discrete.apply(lambda x: x.hour) * 4 + data.syscreatedate_discrete.apply(lambda x: x.minute) / 15
data["reservationfrom_daytime"] = data.reservationfrom_discrete.apply(lambda x: x.hour) * 4 + data.reservationfrom_discrete.apply(lambda x: x.minute) / 15
data["reservationto_daytime"] = data.reservationto_discrete.apply(lambda x: x.hour) * 4 + data.reservationto_discrete.apply(lambda x: x.minute) / 15
data["syscreatedate_discrete_date"] = pd.to_datetime(data['syscreatedate_discrete']).dt.date 
data["reservationfrom_discrete_date"] = pd.to_datetime(data['reservationfrom']).dt.date 
data["drive_firststart_discrete_date"] = pd.to_datetime(data['drive_firststart']).dt.date 
data["booking_trip_same_day"] = data["reservationfrom_discrete_date"] == data["syscreatedate_discrete_date"] 
data["syscreatedate_daytime_endofday"] = data["syscreatedate_daytime"]
data["reservationfrom_daytime_endofday"] = data["reservationfrom_daytime"]
data["reservationto_daytime_endofday"]  = data["reservationto_daytime"] 

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime"] = 0

cond = data["reservationfrom_discrete_date"] <= data["syscreatedate_discrete_date"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = 96

cond = data["reservationfrom_discrete_date"] > data["syscreatedate_discrete_date"] + pd.Timedelta(days=1)
data.loc[cond, "syscreatedate_daytime_endofday"] = 0

cond = data["reservationfrom_daytime_endofday"] == 0
data.loc[cond, "reservationfrom_daytime_endofday"] = 96


cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime_endofday"] = data["reservationfrom_daytime_endofday"]


cond = data["reservationfrom_discrete"] < data["syscreatedate_discrete"] 
data.loc[cond, "syscreatedate_daytime"] = data["reservationfrom_daytime"]

# no energy consumption during service trips
data["required_soc"] = 0

data.to_sql("service_reservations_discrete", engine, schema="msc_2023_dominik", if_exists='replace')
data.head()

## JOIN both tables 
sql1 = """SELECT * FROM msc_2023_dominik.reservations_no_service_discrete"""
reservations_no_service = pd.read_sql(sql1, engine).drop(["level_0"], axis = 1)

sql2 = """SELECT * FROM msc_2023_dominik.service_reservations_discrete"""
reservations_service = pd.read_sql(sql2, engine).drop(["level_0"], axis = 1)

union_df = pd.concat([reservations_no_service, reservations_service], axis=0)
union_df.to_sql("reservations_discrete", engine, schema="msc_2023_dominik", if_exists='replace')


884

In [40]:
### create indices for fast data loading in environment ###
conn = psycopg2.connect(dbname=credentials['database_name'], user=credentials['username'], password=credentials['password'], host=credentials['host'])
cur = conn.cursor()
#cur.execute("CREATE INDEX reservationfrom_discrete_date ON msc_2023_dominik.reservations_discrete USING hash(reservationfrom_discrete_date)")
#cur.execute("CREATE INDEX reservationfrom_discrete_index_b_tree ON msc_2023_dominik.reservations_discrete (reservationfrom_discrete)")
#for i in range(0,83):
#    cur.execute("CREATE INDEX vehicle_no_b_tree_index_week_{} ON msc_2023_dominik.discrete_weeks_{} (vehicle_no)".format(i,i))
    
conn.commit()
cur.close()
conn.close()

In [8]:
### pre-processing of station tables: ###
# Contains all stations in the mobility.station table with valid geometry, that were used during the 
# observation period for bookings (mobility.reservation and mobility.service_reservation)

sql = "WITH stations AS (SELECT DISTINCT start_station_no FROM (SELECT DISTINCT start_station_no FROM mobility.reservation UNION SELECT DISTINCT start_station_no FROM mobility.service_reservation) AS station_union) SELECT station_no, ST_Transform(ST_SetSRID(geom,4326), 2056) AS geom FROM stations LEFT OUTER JOIN mobility.station ON start_station_no = station_no WHERE geom is not NULL"
gdf = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
gdf.to_postgis("distinct_stations", engine, schema="msc_2023_dominik", if_exists='replace')  

In [3]:
### pre-processing of vehicle tables: ###
# get information about each car used in the discrete tables
# replace some car types, which are not used anymore (Budget Electro by Budget, Combi Electro by Combi)
sql = """SELECT vehicle_no, mobility.simulated_ev.model_name, mobility.simulated_ev.brand_name, charge_power, battery_capacity, range, vehicle_category 
         FROM mobility.simulated_ev 
         FULL OUTER JOIN mobility.vehicle USING (vehicle_no) 
         RIGHT OUTER JOIN discrete.discrete_weeks_0 using (vehicle_no)"""
data = pd.read_sql(sql, engine)
data = data.replace(["Budget Electro"], 'Budget')
data = data.replace(['Combi Electro'], 'Combi')
data.to_sql("vehicles_preprocessing", engine, schema="msc_2023_dominik", if_exists='replace')

### vehicle types: ###
# get car information for each category
sql = """WITH vehicles AS (
                            SELECT *
                            FROM msc_2023_dominik.vehicles_preprocessing 
        )
        SELECT model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        FROM vehicles
        WHERE model_name IS NOT NULL
        GROUP by model_name, brand_name, charge_power, battery_capacity, range, vehicle_category 
        ORDER BY vehicle_category"""
data = pd.read_sql(sql, engine)
data.to_sql("vehicles_types", engine, schema="msc_2023_dominik", if_exists='replace')

### Vehicle Information table: ###
# create final vehicle information table; containing all vehicle informatino for each car in the discrete tables
sql = """
WITH vehicles AS (
    SELECT vehicle_no, vehicle_category
    FROM msc_2023_dominik.vehicles_preprocessing 
), 
vehicle_categories AS (
    SELECT *
    FROM msc_2023_dominik.vehicles_types
)
SELECT *
FROM vehicles JOIN vehicle_categories USING(vehicle_category)
"""
data = pd.read_sql(sql, engine)
data = data.drop(["index"], axis = 1)
data.to_sql("vehicle_information", engine, schema="msc_2023_dominik", if_exists='replace')

420

In [2]:
class CarsharingEnv:
    def __init__(self, stations, vehicle_information, episode_len = 24, dt=0.25, 
                 cancellation_penalty = 100, penalty_per_kwh = 0.25, v2g = True, 
                 v2g_demand_event = 500, v2g_max_duration = 3.0, v2g_penalty = 10000, 
                 v2g_probability_charging_event = 0.5, v2g_probability_discharging_event = 0.5, 
                 v2g_morning_time_period = [6.0, 9.0, 10.75], v2g_noon_time_period = [11.0, 14.0, 16.0], 
                 v2g_evening_time_period = [16.25, 19.0, 24.0], v2g_reward = 2000,
                 planned_bookings = True, max_distance_car_assingment = 5000, plot_state = True):
        """
        Parameters
        ----------
        stations: Geopandas Geodataframe
            Locations of car-sharing stations, including a distinct "station_no" attribute with station ID.
        vehicle_information: Pandas Dataframe
            Includes the features "vehicle_no", "charge_power", "battery_capacity", and "vehicle_category" for each car.
        episode_len: int, optional
            Length of one episode in hours, by default 24.
        dt: float, optional
            Time step size in hours, by default 0.25 (a quarter hour).
        cancellation_penalty: int, optional
            Maximum penalty in CHF for a booking cancelled due to not enough charged battery, by default 100.
        penalty_per_kwh: int, optional
            Penalty in CHF/kWh for total negative energy difference between beginning and ending of episode, by default 0.25.
        v2g: boolean, optional
            Boolean indicating if V2G events take place, by default True.
        v2g_demand_event: int, optional
            Energy demand during V2G event in kWh per time step (dt), by default 500.
        v2g_max_duration: int, optional
            Maximum duration of V2G charging or discharging event in hours, by default 3.
        V2G_penalty: int, optional
            Penalty in CHF if agent charges/discharges less energy than specified in "v2g_demand_event" during V2G event, by default 10000.
        v2g_probability_charging_event: float, optional
            Probability that a charging event will take place around noon, by default 0.5.
        v2g_probability_discharging_event: float, optional
            Probability that a discharging event will take place in the morning or evening, by default 0.5.
        v2g_morning_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g discharging event in the morning, by default 6.0 (6 AM).
                             2) last possible time (hours) for starting v2g discharging event in the morning, by default 9.0 (9 AM).
                             3) last possible timestamp for v2g discharging operation, by default 10.75 (10:45 AM).
        v2g_noon_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g charging event at noon, by default 11.0 (11 AM).
                             2) last possible time (hours) for starting v2g charging event at noon, by default 14.0 (2 PM).
                             3) last possible timestamp for v2g charging operation, by default 16.0 (4 PM).
        v2g_evening_time_period: list, optional
            List containing: 1) first possible time (hours) for starting v2g discharging event in the evening, by default 16.25 (4:15 PM).
                             2) last possible time (hours) for starting v2g discharging event in the evening, by default 19.0 (7 PM).
                             3) last possible timestamp for v2g charging operation, by default 24 (00:00 AM, next day).
        v2g_reward: int, optional
            Revenue during v2g operations per timestep in CHF, by default 2000 CHF.
        planned_bookings: boolean, optional
            Boolean indicating whether there are planned bookings in the environment; otherwise, all bookings are spontaneous, by default True.
        max_distance_car_assingment: int, optional
            Maximum search distance in meter for Car-assingment problem, by default 5000.
        plot_state: boolean, optional
            Plot current state of enviorment or not, by defualt True.
        ----------
        Observation space: 5 parts:
        1) Location for each car (three options):
            - Station number (1000-5000)
            - Reservation number (2x'xxx'xxx) during trip
            - Reservation number (3x'xxx'xxx) during relocation
            - -1 if car is not available
        2) State of charge (SOC) for each vehicle (between 0 and 1).
        3) Timestamp of the next planned booking for each car (discrete between 0 and self.episode_len).
        4) Duration of the next planned booking (measured in the number of time steps of length self.dt).
        5) Binary variable indicating the occurrence of a vehicle-to-grid (V2G) event.
        Remark: States 3) and 4) are only included if the variable "planned_bookings" is True.
        ----------
        Action space: 3 actions for each car:
        1) 0 = do nothing.
        2) 1 = charging.
        3) 3 = discharging (V2G).
        """

        # environment settings
        self.dt = dt
        self.episode_len = int(episode_len / self.dt) 
        self.planned_bookings = planned_bookings
        self.max_distance_car_assingment = max_distance_car_assingment
        self.plot_state = plot_state
        
        # stations in system
        self.stations = stations
        
        # vehicle information
        self.vehicles_id = vehicle_information["vehicle_no"]
        self.nr_vehicles = len(self.vehicles_id)
        self.chariging_power = vehicle_information["charge_power"]
        self.battery_capacities = vehicle_information["battery_capacity"] 
        self.vehicle_type = vehicle_information["vehicle_category"] 
        
        # define state boundaries for slicing list
        # locations upper bound
        self.locations_upper = self.nr_vehicles
        # soc upper bound
        self.soc_upper = 2*self.nr_vehicles
        # locations, upper bound:
        self.reservation_time_upper = self.nr_vehicles * 3
        self.v2g_lower = self.nr_vehicles * 4
        
        # Save discrete planned reservations
        self.planned_reservations_df_res = pd.DataFrame()
        self.planned_reservations_df_dur = pd.DataFrame()
            
        
    def reset(self, daily_data, reservations, week_nr, day):
        """
        Parameters
        ----------
        daily_data: Pandas DataFrame
            Contains the car trips over the day.
        reservations: Pandas DataFrame
            Contains information about the cars, including "vehicle_no", "charge_power", "battery_capacity", and "vehicle_category".

        Returns
        ----------
        self.state: numpy ndarray
            The reset state of the environment at the first time step. It includes the following information for each car:
            1) Location:
                - Station number (1000-5000)
                - Reservation number (2x'xxx'xxx) during trip
                - Reservation number (3x'xxx'xxx) during relocation
                - -1 if the car is not available
            2) State of charge (SOC) for each vehicle (between 0 and 1).
            3) Timestamp of the next planned booking for each car (discrete value between 0 and self.episode_len).
            4) Duration of the next planned booking (measured in the number of time steps of length self.dt).
            5) Binary variable indicating the occurrence of a vehicle-to-grid (V2G) event.
            Note: The information in 3) and 4) is included only if the variable "planned_bookings" is True.
        """
        
        # set time to 0
        self.t = 0
        
        if week_nr == 0 and day == 98:
            self.week_nr = week_nr
            self.planned_reservations_df_res = pd.DataFrame()
            self.planned_reservations_df_dur = pd.DataFrame()
            self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
            self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id
        
        
        if week_nr != self.week_nr:
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                self.planned_reservations_df_res.to_sql("planned_reservations_discrete_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='replace')
                self.planned_reservations_df_dur.to_sql("planned_durations_discrete_{}".format(week_nr-1), engine, schema="msc_2023_dominik", if_exists='replace')
                self.planned_reservations_df_res = pd.DataFrame()
                self.planned_reservations_df_res["vehicle_no"] = self.vehicles_id
                self.planned_reservations_df_dur = pd.DataFrame()
                self.planned_reservations_df_dur["vehicle_no"] = self.vehicles_id
        # set week number
        self.week_nr = week_nr
        # reset reward lists 
        self.reward_list_trips = []
        self.reward_list_charging = []
        self.reward_list_cancellation_penalty = []
        self.reward_list_v2g = []
        self.reward_list = []
        
        # reset state
        self.state_old = np.zeros(self.nr_vehicles * 4 + 1)
        
        # reset planned reservations:
        self.planned_reservations_car = {key: [] for key in self.vehicles_id}
        
        
        
        ### initialize state at t = 0 ###
        
        # 1) car locations (three options)
        car_locations = daily_data.iloc[:,0].values
        
        # 2) SOC (state of charge) ##
        #car_SOC = np.random.rand(self.nr_vehicles) # random values between 0-100%
        #car_SOC = np.zeros(self.nr_vehicles) # all car batteries empty (0%)
        #car_SOC = np.full((self.nr_vehicles,), 0.2) # all car batteries at 20%
        #car_SOC = np.random.uniform(low=0.5, high=1, size=self.nr_vehicles) # All cars randomly between 50-100%
        np.random.seed(42)
        rng = np.random.RandomState(42)
        car_SOC = rng.uniform(low=0.5, high=1, size=self.nr_vehicles)
        
         # state 5) Binary V2G event
        v2g_event = np.array([0])
 
        # final state with planned bookings
        
        next_reservation, duration_next_reservation = self.update_reservation_state(reservations, True)
            
            # concatinate states 1-5
        self.state = np.concatenate([car_locations, car_SOC, next_reservation, duration_next_reservation, v2g_event])
        
        


        return self.state

   
    
    
    def update_reservation_state(self, reservations, reset):
        """
         Parameters
        ----------
        reservations: Pandas DataFrame
            Includes the features "syscreatedate_daytime", "vehicle_no", "reservationfrom_daytime", and "reservation_duration" for each reservation.
        reset: boolean, optional
            Boolean indicating whether to reset the environment or not.

        Returns
        ----------
        next_reservation : numpy ndarray
            Timestamp of the next planned booking for each car (discrete value between 0 and self.episode_len).
        duration_next_reservation : numpy ndarray
            Duration of the next planned booking (measured in the number of time steps of length self.dt).
        """

        # initalize variables if during reset of environment
        if reset is True:
            time_ = 0
            next_reservation = np.ones(self.nr_vehicles) * -1
            duration_next_reservation = np.ones(self.nr_vehicles) * -1

        # get current state of next planned reservations (timestamp and duration)
        else:
            time_ = self.t + 1
            next_reservation = self.state[self.soc_upper : self.reservation_time_upper]
            duration_next_reservation = self.state[self.reservation_time_upper :self.v2g_lower]

        # filter reservations: 
        # bookings before t = 0 have value syscreatedate_daytime = 0
        reservations_t =  reservations[reservations["syscreatedate_daytime"] == time_]


        # iterate over all vehicles, search planned reservations
        count = 0

        for vehicle_id in self.vehicles_id:  
            
            # get reservation of car
            current_car = reservations_t[reservations_t["vehicle_no"] == vehicle_id]
            # remove reservations in past
            if self.planned_reservations_car[vehicle_id] and int(self.planned_reservations_car[vehicle_id][0][0]) <= self.t:
                self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]
            # skip if no new reservation found
            if current_car.empty:
                # if current state's reservation is in the past 
                if next_reservation[count] <=  self.t and next_reservation[count] != -1:
                    # check for planned reservations saved in the past
                    if self.planned_reservations_car[vehicle_id]:
                        # save reservation timestamp
                        next_reservation[count] = self.planned_reservations_car[vehicle_id][0][0]
                        # save reservation duration
                        duration_next_reservation[count] = self.planned_reservations_car[vehicle_id][0][1]
                        # remove reservation from dict with planned reservations
                        self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]
                    else:
                        # assign -1 for no planned reservation
                        next_reservation[count] = -1
                        duration_next_reservation[count] = -1
                count +=1
                continue

                

            for i in range(0,len(current_car)):    

                # get timestamp and duration of next planned reservation
                reservation_time = current_car["reservationfrom_daytime"].iloc[i]
                reservation_duration = current_car["reservation_duration"].iloc[i]

                # save directly if during reset of environment
                if i == 0 and reset is True:
                    # save timestamp of next reservation
                    next_reservation[count] = reservation_time

                    # save reservation duration
                    duration_next_reservation[count] = reservation_duration


                if reset is True:
                    # save directly first reseration during reset of environment
                    if i == 0 and reset is True:
                        # save timestamp of next reservation
                        next_reservation[count] = reservation_time

                        # save reservation duration
                        duration_next_reservation[count] = reservation_duration

                    # save other found reservation during reset of environment in dict
                    else:
                        new_reservation = [reservation_time, reservation_duration]

                        # remain order in dict (nearest reservation in first index)
                        index = bisect.bisect_left([sublist[0] for sublist in  self.planned_reservations_car[vehicle_id]], new_reservation[0])

                        # save timestamp and duration of new but later reservations in dict
                        self.planned_reservations_car[vehicle_id].insert(index, new_reservation)

                else: 
                    # save found reservation environment in dict
                    new_reservation = [reservation_time, reservation_duration]

                    # remain order in dict (nearest reservation in first index)
                    index = bisect.bisect_left([sublist[0] for sublist in  self.planned_reservations_car[vehicle_id]], new_reservation[0])

                    # save timestamp and duration of new but later reservations in dict
                    self.planned_reservations_car[vehicle_id].insert(index, new_reservation)


            # update only if reservation is before last planned reservation
            if  reset is False and self.planned_reservations_car[vehicle_id][0][0] <  next_reservation[count]:
                # save timestamp of next reservation
                next_reservation[count] = self.planned_reservations_car[vehicle_id][0][0]

                # save reservation duration
                duration_next_reservation[count] = self.planned_reservations_car[vehicle_id][0][1]

                # remove reservation from dict
                self.planned_reservations_car[vehicle_id] = self.planned_reservations_car[vehicle_id][1:]

            count += 1
        
        self.planned_reservations_df_res[daily_data.columns[time_]] = next_reservation
        self.planned_reservations_df_dur[daily_data.columns[time_]] = duration_next_reservation

        return next_reservation, duration_next_reservation


    def step(self, daily_data, reservations):

        next_reservation, duration_next_reservation = self.update_reservation_state(reservations, False)
        self.state[self.soc_upper : self.reservation_time_upper] = next_reservation
        self.state[self.reservation_time_upper :self.v2g_lower] = duration_next_reservation

        done = True if self.t == (self.episode_len - 2) else False
        # update time step
        self.t += 1

        return self.state, done, {}



In [3]:
import warnings
warnings.filterwarnings('ignore')

# get station geodata, create spatial index
sql = " SELECT * FROM msc_2023_dominik.distinct_stations"
stations = gpd.read_postgis(sql, engine, geom_col='geom',crs = "EPSG:2056")
stations.sindex

# get vehicle data
sql = "SELECT * FROM msc_2023_dominik.vehicle_information ORDER BY vehicle_no"
vehicles = pd.read_sql(sql, engine)


env = CarsharingEnv(stations, vehicles, planned_bookings = True, plot_state = False)

# number of days to simulate 
nr_iterations = 577

# number of vehicles
nr_vehicles = len(vehicles)

# maximal simulation length
if nr_iterations > 577:
    nr_iterations = 577
    
count = 0

# iterate over weeks (for loading weekly discrete data)
for week_nr in range(0, math.ceil(nr_iterations / 7)):
    # load discrete car-sharing table
    sql =  "SELECT * FROM discrete.discrete_weeks_{} ORDER BY vehicle_no".format(week_nr)
    data = pd.read_sql(sql, engine)
        
    # iteration for each day
    for day in range(98,676,96):
        # all requested days are simulated
        if count == nr_iterations:
            s = env.reset(daily_data, reservations, 83, day)
            break
            
        # get date
        date = pd.to_datetime(data.columns[day-97])
       
        # load reservations of current day, create indices
        sql = "SELECT * FROM msc_2023_dominik.reservations_discrete WHERE reservationfrom_discrete_date = '{}' or drive_firststart_discrete_date = '{}' ORDER BY reservationfrom_discrete".format(date, date)
        reservations = pd.read_sql(sql, engine)
        reservations.set_index(['vehicle_no', 'reservationfrom_daytime', 'syscreatedate_daytime'], inplace=True, drop=False)

        # get discete data of day
        daily_data = data.iloc[:,day-97:day]   
        
        
        # reset day at begining of new episode (day)
        s = env.reset(daily_data, reservations, week_nr, day)
        
        # simulate day in 15 min steps
        done = False
        counter = 0
        while not done:


            # proceed one time step
            s, done, _ = env.step(daily_data, reservations)
        
            counter +=1
        print("Week: ", week_nr," Day: ", day)
        count += 1

Week:  0  Day:  98
Week:  0  Day:  194
Week:  0  Day:  290
Week:  0  Day:  386
Week:  0  Day:  482
Week:  0  Day:  578
Week:  0  Day:  674
Week:  1  Day:  98
Week:  1  Day:  194
Week:  1  Day:  290
Week:  1  Day:  386
Week:  1  Day:  482
Week:  1  Day:  578
Week:  1  Day:  674
Week:  2  Day:  98
Week:  2  Day:  194
Week:  2  Day:  290
Week:  2  Day:  386
Week:  2  Day:  482
Week:  2  Day:  578
Week:  2  Day:  674
Week:  3  Day:  98
Week:  3  Day:  194
Week:  3  Day:  290
Week:  3  Day:  386
Week:  3  Day:  482
Week:  3  Day:  578
Week:  3  Day:  674
Week:  4  Day:  98
Week:  4  Day:  194
Week:  4  Day:  290
Week:  4  Day:  386
Week:  4  Day:  482
Week:  4  Day:  578
Week:  4  Day:  674
Week:  5  Day:  98
Week:  5  Day:  194
Week:  5  Day:  290
Week:  5  Day:  386
Week:  5  Day:  482
Week:  5  Day:  578
Week:  5  Day:  674
Week:  6  Day:  98
Week:  6  Day:  194
Week:  6  Day:  290
Week:  6  Day:  386
Week:  6  Day:  482
Week:  6  Day:  578
Week:  6  Day:  674
Week:  7  Day:  98
Week:  7